In [1]:
import itertools as itt

import joblib as jl
import pathlib as pl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from IPython.display import display
from src.root_path import config_path
from configparser import ConfigParser

[nems.configs.defaults INFO] Saving log messages to /tmp/nems/NEMS 2021-09-13 092422.log
/auto/users/mateo/nems_db/nems_lbhb/SettingXML.py:66: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  {K: SubEl.get(K) for K in SubEl.keys() if K is not 'name'}
/auto/users/mateo/nems_db/nems_lbhb/SettingXML.py:76: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  {K: SubEl.get(K) for K in SubEl.keys() if K is not 'name'}
/auto/users/mateo/nems_db/nems_lbhb/SettingXML.py:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  {K: SubEl.get(K) for K in SubEl.keys() if K is not 'name'}


importing region_map: site ley058d has undefined region


[numexpr.utils INFO] NumExpr defaulting to 8 threads.


In [2]:
config = ConfigParser()
config.read_file(open(config_path / 'settings.ini'))

In [3]:
#general plottin formating
plt.style.use(['dark_background', config_path / 'notebook.mplstyle'])


In [4]:
# 4 sound file
DF4_file = pl.Path(config['paths']['analysis_cache']) /'210302_consolidated_summary_DF_alpha_0.05/dprime_absolute-None_montecarlo-1000_raster_fs-30_reliability-0.1_smoothing_window-0_zscore-True'
DF4 = jl.load(DF4_file)

# 10 soundfile
DF10_file = pl.Path(config['paths']['analysis_cache']) /'210902_consolidated_summary_DF_alpha_0.05/dprime_absolute-None_montecarlo-1000_raster_fs-30_reliability-0.1_smoothing_window-0_stim_type-permutations_zscore-True'

DF10 = jl.load(DF10_file)


In [5]:
mult_comp_cont = 4
# pulls DF, removes unused columns and data, refactor redundant id columns and create probe_id
def format_dataframe(DF):
    ff_probe = DF.probe != 'mean'
    ff_pairs = DF.context_pair != 'mean'
    try:
        ff_mean = DF.mean_signif_type == 'shuffles'
    except:
        print('no mean_signif_type, taking all values')
        ff_mean = pd.Series(np.full(DF.shape[0], True))


    try:
        ff_stim = DF.stim_type == 'permutations'
    except:
        print('no stim_type, taking all values')
        ff_stim = pd.Series(np.full(DF.shape[0], True))


    ff_analylis = DF.analysis.isin(['SC', 'fdPCA'])
    # ff_corr = DF.mult_comp_corr == 'consecutive_3'
    ff_corr = DF.mult_comp_corr == f'consecutive_{mult_comp_cont}'
    ff_metric = DF.metric.isin(['significant_abs_mass_center', 'significant_abs_sum'])

    good_cols =['analysis', 'mult_comp_corr', 'region', 'siteid',  'cellid', 'context_pair',
                'probe', 'metric', 'value']
    filtered = DF.loc[ff_stim & ff_mean & ff_pairs & ff_probe & ff_analylis & ff_corr & ff_metric, good_cols]

    filtered['probe'] = [int(p) for p in filtered['probe']]
    filtered['context_pair'] = [f"{int(cp.split('_')[0]):02d}_{int(cp.split('_')[1]):02d}"
                                for cp in filtered['context_pair']]

    # rename metrics and analysis for ease of ploting
    filtered['metric'] = filtered['metric'].replace({'significant_abs_mass_center': 'center of mass (ms)',
                                                     'significant_abs_mean': "mean d'",
                                                     'significant_abs_sum': "integral (d'*ms)"})
    filtered['analysis'] = filtered['analysis'].replace({'SC': 'single cell',
                                                         'fdPCA': 'population',
                                                         'pdPCA': 'probewise pop',
                                                         'LDA': 'pop ceiling'})

    filtered['id'] = filtered['cellid'].fillna(value=filtered['siteid'])
    filtered = filtered.drop(columns=['cellid', 'siteid'])

    filtered['value'] = filtered['value'].fillna(value=0)

    # permutation related preprocesing.
    # creates a new column relating probe with  context pairs
    ctx = np.asarray([row.split('_') for row in filtered.context_pair], dtype=int)
    prb = np.asarray(filtered.probe, dtype=int)

    silence = ctx == 0
    same = ctx == prb[:,None]
    different = np.logical_and(~silence, ~same)

    name_arr = np.full_like(ctx, np.nan, dtype=object)
    name_arr[silence] = 'silence'
    name_arr[same] = 'same'
    name_arr[different] = 'diff'
    comp_name_arr = np.apply_along_axis('_'.join, 1, name_arr)

    # swaps clasification names to not have repetitions i.e. diff_same == same_diff
    comp_name_arr[np.where(comp_name_arr == 'same_silence')] = 'silence_same'
    comp_name_arr[np.where(comp_name_arr == 'diff_silence')] = 'silence_diff'
    comp_name_arr[np.where(comp_name_arr == 'diff_same')] = 'same_diff'
    comp_name_arr[np.where(comp_name_arr == 'same_silence')] = 'silence_same'

    filtered['trans_pair'] = comp_name_arr

    ord_cols = ['analysis', 'region', 'id', 'context_pair', 'trans_pair', 'probe', 'metric', 'value']
    pivot_idx = [col for col in ord_cols if col not in ['value', 'metric']]
    pivoted = filtered.pivot_table(index=pivot_idx, columns='metric', values='value', aggfunc='first').reset_index()

    full_long = filtered # saves long format for subsamplig analysis

    return pivoted, full_long

pivoted, full_long = pd.DataFrame(), pd.DataFrame()
for df, nsounds in zip([DF4, DF10], [4, 10]):
    pvtd, fl_lg = format_dataframe(df)
    pvtd['nsounds'] = nsounds
    fl_lg['nsounds'] = nsounds
    pivoted = pivoted.append(pvtd)
    full_long = full_long.append(fl_lg)

del(fl_lg, pvtd)
display(full_long)

no mean_signif_type, taking all values
no stim_type, taking all values


,analysis,mult_comp_corr,region,context_pair,probe,metric,value,id,trans_pair,nsounds
75735,single cell,consecutive_4,PEG,00_01,1,center of mass (ms),0.0,AMT028b-01-1,silence_same,4
75736,single cell,consecutive_4,PEG,00_01,1,center of mass (ms),0.0,AMT028b-01-2,silence_same,4
75737,single cell,consecutive_4,PEG,00_01,1,center of mass (ms),0.0,AMT028b-04-1,silence_same,4
75738,single cell,consecutive_4,PEG,00_01,1,center of mass (ms),0.0,AMT028b-06-1,silence_same,4
75739,single cell,consecutive_4,PEG,00_01,1,center of mass (ms),0.0,AMT028b-07-1,silence_same,4
...,...,...,...,...,...,...,...,...,...,...
854695,population,consecutive_4,A1,07_09,10,integral (d'*ms),0.0,TNC015a,diff_diff,10
854696,population,consecutive_4,A1,07_10,10,integral (d'*ms),0.0,TNC015a,same_diff,10
854697,population,consecutive_4,A1,08_09,10,integral (d'*ms),0.0,TNC015a,diff_diff,10
854698,population,consecutive_4,A1,08_10,10,integral (d'*ms),0.0,TNC015a,same_diff,10


In [6]:
# counts the total number of values for each comparison (SC, fDPCA) x (sile_diff, sile_same, diff_diff, same_diff)
# counts the proportion of signifciant vs non singificant(zero-zero) values
# decimates the more numerous comparisons??

# set the array to group by id, context_pair, and probe. Keeping analysis, region and transision_pair
# the final array should have analysis, region and trans pair as rowns, and total count, and significants as columns

def nozero_mean(arr):
    arr[arr==0] = np.nan
    return np.nanmean(arr)

def nozero_count(arr):
    return np.sum(arr>0)

def corrected_count(arr):
    n_comparisons = np.size(arr)
    alpha = 0.05**mult_comp_cont * (30 - mult_comp_cont)
    chance = n_comparisons * alpha

    raw_count = np.sum(arr>0)
    if raw_count > chance:
        corr_count = raw_count
    else:
        corr_count = 0
    return corr_count

def nozero_proportion(arr):
    return np.sum(arr>0) / np.size(arr)

def nozero_percentage(arr):
    return np.sum(arr>0) / np.size(arr) *100

def corrected_proportion(arr):
    n_comparisons = np.size(arr)
    alpha = 0.05**mult_comp_cont * (30 - mult_comp_cont)
    chance = n_comparisons * alpha

    raw_count = np.sum(arr>0)
    if raw_count > chance:
        corr_count = raw_count
    else:
        corr_count = 0
    return corr_count / np.size(arr)

filtered = full_long
group_ready = pivoted.set_index(['analysis', 'region', 'trans_pair']).loc[:, filtered.metric.unique()]
agg_funcs = {'center of mass (ms)': [('mean duration', nozero_mean)],
             "integral (d'*ms)": [('mean amplitude', nozero_mean),
                                     ('total', 'size'),
                                     ('n_signif', nozero_count)]}

grouped = group_ready.groupby(['analysis', 'region', 'trans_pair']).agg(agg_funcs)
grouped.columns = grouped.columns.droplevel(0)
grouped['significant %'] = grouped.n_signif / grouped.total * 100

display(grouped)

mean duration  mean amplitude  total  \
analysis    region trans_pair                                           
population  A1     diff_diff        262.126327      295.537675   2352   
                   same_diff        236.317175      307.331638    732   
                   silence_diff     216.244759      379.872559    732   
                   silence_same     208.043779      465.515417    124   
            PEG    diff_diff        115.342923      297.866624    228   
                   same_diff        131.897413      328.627656    228   
                   silence_diff     136.095736      393.258078    228   
                   silence_same     167.972786      429.614277     76   
single cell A1     diff_diff        318.940545      181.916530  93864   
                   same_diff        308.953268      186.413325  27174   
                   silence_diff     289.390279      209.432437  27174   
                   silence_same     285.887404      225.288201   4118   
            PEG    diff_diff        173.334498      225.751467   5400   
                   same_diff        203.595567      225.590667   5400   
                   silence_diff     197.581493      242.205015   5400   
                   silence_same     252.343107      231.156172   1800   

                                 n_signif  significant %  
analysis    region trans_pair                             
population  A1     diff_diff        645.0      27.423469  
                   same_diff        189.0      25.819672  
                   silence_diff     304.0      41.530055  
                   silence_same      52.0      41.935484  
            PEG    diff_diff         60.0      26.315789  
                   same_diff         62.0      27.192982  
                   silence_diff      91.0      39.912281  
                   silence_same      27.0      35.526316  
single cell A1     diff_diff       4235.0       4.511847  
                   same_diff       1125.0       4.139987  
                   silence_diff    1647.0       6.060941  
                   silence_same     245.0       5.949490  
            PEG    diff_diff        232.0       4.296296  
                   same_diff        244.0       4.518519  
                   silence_diff     287.0       5.314815  
                   silence_same     124.0       6.888889

## count number of cells or sites with significant contextual effects

In [7]:
# analyses = ['single cell', 'population', 'probewise pop', 'pop ceiling'] # 4 sounds
analyses = ['single cell', 'population']
nsounds = [4, 10]# 10 sounds

for analysis, ns in itt.product(analyses, nsounds):
    print(f'\n{analysis} {ns} sounds')
    all_id_df= pivoted.loc[(pivoted.nsounds == ns) & (pivoted.analysis == analysis), ['id', "integral (d'*ms)"]]

    # count of cells or sites with significant instances
    all_id_count = len(all_id_df.id.unique())
    good_id_df= all_id_df.loc[(all_id_df["integral (d'*ms)"] > 0), ['id']]
    good_id_count = len(good_id_df.id.unique())
    print(f'{good_id_count} of {all_id_count} total modulated neurons/PCs. {good_id_count/all_id_count*100:.2f}%')

    # total count of significant instances
    total_instances = all_id_df["integral (d'*ms)"].shape[0]
    significant_instances = np.sum(all_id_df["integral (d'*ms)"] > 0)
    print(f"{significant_instances} of {total_instances} total significant instances, {significant_instances/total_instances*100:.2f}%")

    # count of significant instances per neuron or PC, corrected for multiple comparisons
    signif_prop_per_neuron = all_id_df.groupby('id').agg(corr_signif_prop=("integral (d'*ms)",corrected_proportion))
    modulated_neurons =  signif_prop_per_neuron.loc[signif_prop_per_neuron.corr_signif_prop > 0]
    print(f"modulated neurons/PCs have {np.mean(modulated_neurons.values)*100 :.2f}% significant instances on average")

    # count of significant neurons per site
    if analysis == 'single cell':
        signif_prop_per_neuron = signif_prop_per_neuron.reset_index()
        signif_prop_per_neuron['site'] = signif_prop_per_neuron['id'].apply(lambda x: x[:7])
        gp = signif_prop_per_neuron.groupby('site').agg(
            signif_neu_prop=('corr_signif_prop', lambda x: np.sum(x>0)/np.size(x)))
        print(f"sites have {np.mean(gp.values)*100 :.2f}% of modulated neurons on average")


single cell 4 sounds
375 of 862 total modulated neurons/PCs. 43.50%
1463 of 34480 total significant instances, 4.24%
modulated neurons/PCs have 9.75% significant instances on average
sites have 42.27% of modulated neurons on average

single cell 10 sounds
199 of 247 total modulated neurons/PCs. 80.57%
6676 of 135850 total significant instances, 4.91%
modulated neurons/PCs have 6.10% significant instances on average
sites have 78.95% of modulated neurons on average

population 4 sounds
35 of 35 total modulated neurons/PCs. 100.00%
428 of 1400 total significant instances, 30.57%
modulated neurons/PCs have 30.57% significant instances on average

population 10 sounds
6 of 6 total modulated neurons/PCs. 100.00%
1002 of 3300 total significant instances, 30.36%
modulated neurons/PCs have 30.36% significant instances on average
